In [4]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import math

%matplotlib inline

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D,Input
from keras.optimizers import SGD,Adam
from skimage.measure import compare_ssim as ssim

Using TensorFlow backend.


In [29]:
# Define function to determine peak signal-to-noise ratio

def pnsr(target,ref):
    
    #Convert Data Type of Images
    target_data = target.astype(float)
    ref_data = ref.astype(float)
    
    diff = ref_data - target_data
    diff=diff.flatten('C')
    print(diff)
    rmse = math.sqrt(np.mean(diff **2.))
    
    return  20* math.log10(255. /rmse)

In [30]:
# Define function to determine mean squared error

def mse(target,ref):
    target_data = target.astype(float)
    ref_data = ref.astype(float)
    
    err= np.sum((target_data -ref_data)**2)
    err /= float(target.shape[0]*target.shape[1])
    
    return err
    

In [7]:
# Function to compare all 3 image quality metrics
def compare_images(target,ref):
    scores=[]
    scores.append(pnsr(target,ref))
    scores.append(mse(target,ref))
    scores.append(ssim(target,ref,multichannel=True))
    
    return scores

In [33]:
# Prepare Images
def prepare_images(path,factor):
    
    #loop through files
    for file in os.listdir(path):
        # Open the file
        
        img=cv2.imread(path+'/'+file)
    
    #find image dimensions
        h,w,c = img.shape
        nh= h//factor
        nw= w//factor
    
    # resize image down
        img=cv2.resize(img,(nw,nh),interpolation=cv2.INTER_LINEAR)
    
     # resize image UP
        img=cv2.resize(img,(w,h),interpolation=cv2.INTER_LINEAR)
    
    #save the file
    
        print("Saving {}".format(file))
        cv2.imwrite('images/{}'.format(file),img)
    
    

In [34]:
prepare_images('source/',2)

Saving baboon.bmp
Saving baby_GT.bmp
Saving barbara.bmp
Saving bird_GT.bmp
Saving butterfly_GT.bmp
Saving coastguard.bmp
Saving comic.bmp
Saving face.bmp
Saving flowers.bmp
Saving foreman.bmp
Saving head_GT.bmp
Saving lenna.bmp
Saving monarch.bmp
Saving pepper.bmp
Saving ppt3.bmp
Saving woman_GT.bmp
Saving zebra.bmp


In [36]:
# Testing Image qualities

for file in os.listdir('images/'):
    # Read the Images
    target=cv2.imread('images/{}'.format(file))
    ref=cv2.imread('source/{}'.format(file))
    
    # The Scores
    scores=compare_images(target,ref)
    
    print("{}\n  PNSR: {}\n  MSE: {}\n  SSIM: {}\n".format(file,scores[0],scores[1],scores[2]))

[-12. -61. -57. ... -17. -17. -22.]
baboon.bmp
  PNSR: 22.157084083442548
  MSE: 1187.1161333333334
  SSIM: 0.629277587900277

[0. 0. 0. ... 0. 0. 0.]
baby_GT.bmp
  PNSR: 34.37180640966199
  MSE: 71.28874588012695
  SSIM: 0.9356987872724932

[1. 0. 0. ... 0. 0. 1.]
barbara.bmp
  PNSR: 25.906629837568126
  MSE: 500.65508535879627
  SSIM: 0.8098632646406401

[ 1.  1.  0. ... -1.  0.  2.]
bird_GT.bmp
  PNSR: 32.896644728720005
  MSE: 100.12375819830247
  SSIM: 0.9533644866026473

[ 4. -1. -5. ...  0.  0. -1.]
butterfly_GT.bmp
  PNSR: 24.782076560337416
  MSE: 648.6254119873047
  SSIM: 0.8791344763843051

[-36. -45. -46. ...   0.  -1.   0.]
coastguard.bmp
  PNSR: 27.161600663887082
  MSE: 375.00887784090907
  SSIM: 0.756950063354931

[-11. -11. -11. ... -13. -17. -26.]
comic.bmp
  PNSR: 23.799861502225532
  MSE: 813.2338836565096
  SSIM: 0.8347335416398209

[0. 0. 0. ... 4. 1. 7.]
face.bmp
  PNSR: 30.99220650287191
  MSE: 155.23189718546524
  SSIM: 0.8008439492289884

[-2. -2. -2. ... -1. 

In [19]:
for file in os.listdir('source/'):
    print(file)

baboon.bmp
baby_GT.bmp
barbara.bmp
bird_GT.bmp
butterfly_GT.bmp
coastguard.bmp
comic.bmp
face.bmp
flowers.bmp
foreman.bmp
head_GT.bmp
lenna.bmp
monarch.bmp
pepper.bmp
ppt3.bmp
woman_GT.bmp
zebra.bmp


In [27]:
it=cv2.imread("images/zebra.bmp")

In [28]:
cv2.imshow("1",it)
cv2.waitKey()
cv2.destroyAllWindows()

In [37]:
def model():
    SRCNN= Sequential()
    #add layers
    SRCNN.add(Conv2D(filters=128, kernel_size=(9,9),kernel_initializer='glorot_uniform',activation='relu',padding='valid',
                     use_bias=True, input_shape=(None,None,1)))
    SRCNN.add(Conv2D(filters=64, kernel_size=(3,3),kernel_initializer='glorot_uniform',activation='relu',padding='same',
                     use_bias=True, input_shape=(None,None,1)))
    SRCNN.add(Conv2D(filters=1, kernel_size=(5,5),kernel_initializer='glorot_uniform',activation='linear',padding='valid',
                     use_bias=True, input_shape=(None,None,1)))
    # define optimizer
    adam = Adam(lr=0.003)
    
    #Compile Model
    SRCNN.compile(optimizer=adam,loss='mean_squared_error',metrics=['mean_squared_error'])
    
    return SRCNN
    
    